<a href="https://colab.research.google.com/github/cuie23/learning-pytorch/blob/main/torch_vision_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
import requests
from pathlib import Path

# Download helper functions from Learn PyTorch repo (if not already downloaded)
if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download")
else:
  print("Downloading helper_functions.py")
  # Note: you need the "raw" GitHub URL for this to work
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

In [5]:
train_data = datasets.FashionMNIST(
    root="data",           # where to download data to?
    train=True,            # get training data
    download=True,         # download data if it doesn't exist on disk
    transform=ToTensor()   # images come as PIL format, turn into Torch tensors
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False, # get test data
    download=True,
    transform=ToTensor()
)

100%|██████████| 26.4M/26.4M [00:02<00:00, 12.0MB/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 200kB/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.74MB/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 6.10MB/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [6]:
train_data.data.shape, train_data.targets.shape

(torch.Size([60000, 28, 28]), torch.Size([60000]))

In [7]:
# Create DataLoaders

BATCH_SIZE = 64

train_dataloader = DataLoader(
    train_data,
    batch_size = BATCH_SIZE,
    shuffle = True
)

test_dataloader = DataLoader(test_data,
    batch_size=BATCH_SIZE,
    shuffle=False     # don't necessarily have to shuffle the testing data
)

In [8]:
def accuracy_fn(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
  acc = (correct / len(y_pred)) * 100
  return acc

## Normal NN

In [9]:
class FashionMNISTModel(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
    super().__init__()

    self.layer_stack = nn.Sequential(
        nn.Flatten(),    # Convert 2d image data into vector
        nn.Linear(in_features = input_shape, out_features = hidden_units),
        nn.Linear(in_features=hidden_units, out_features=output_shape)
    )

  def forward(self, x):
    return self.layer_stack(x)

## CNN

In [10]:
class CNN(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
    super().__init__()

    # Convolution/Pooling
    self.block_1 = nn.Sequential(
        nn.Conv2d(in_channels = input_shape,
                  out_channels = hidden_units,
                  kernel_size = 3,    # dimension of kernel/filter
                  stride = 1,         # number of steps kernel shifts on each step
                  padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, # Max pooling
                     stride = 2)
    )

    # Convolution/Pooling
    self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

    # Dense layer
    self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*7*7,       # Size from how network compresses/changes shape of input data
                      out_features=output_shape)
        )

  def forward(self, x):
    x = self.block_1(x)
    x = self.block_2(x)
    x = self.classifier(x)
    return x

In [11]:
torch.manual_seed(42)

# Need to setup model with input parameters
model = CNN(
    input_shape=1,
    hidden_units=10,
    output_shape=len(train_data.classes)
).to(device)
next(model.parameters()).device

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters(), lr = 0.1)

In [12]:
from tqdm.auto import tqdm  # progress bar

torch.manual_seed(42)

epochs = 2

for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n-------")

  train_loss = 0
  for batch, (X, y) in enumerate(train_dataloader):
    X, y = X.to(device), y.to(device)

    model.train()

    print(X.shape)
    y_pred = model(X)

    loss = loss_fn(y_pred, y)
    train_loss += loss     # accumulate loss across batches

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 400 == 0:
      print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")

    test_loss, test_acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in test_dataloader:
            X, y = X.to(device), y.to(device)

            test_pred = model(X)
            test_loss += loss_fn(test_pred, y) # accumulatively add up the loss per epoch
            test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))

        # Calculations on test metrics need to happen inside torch.inference_mode()
        # Divide total test loss by length of test dataloader (per batch)
        test_loss /= len(test_dataloader)

        # Divide total accuracy by length of test dataloader (per batch)
        test_acc /= len(test_dataloader)

    print(f"Train loss: {train_loss:.5f} | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 0
-------
torch.Size([64, 1, 28, 28])
Looked at 0/60000 samples
Train loss: 2.30835 | Test loss: 2.29914, Test acc: 10.01%
torch.Size([64, 1, 28, 28])
Train loss: 4.60229 | Test loss: 2.29807, Test acc: 10.01%
torch.Size([64, 1, 28, 28])
Train loss: 6.89790 | Test loss: 2.29694, Test acc: 10.01%
torch.Size([64, 1, 28, 28])
Train loss: 9.19448 | Test loss: 2.29583, Test acc: 10.01%
torch.Size([64, 1, 28, 28])
Train loss: 11.49439 | Test loss: 2.29494, Test acc: 10.01%
torch.Size([64, 1, 28, 28])
Train loss: 13.78418 | Test loss: 2.29330, Test acc: 10.01%
torch.Size([64, 1, 28, 28])
Train loss: 16.08313 | Test loss: 2.29151, Test acc: 10.01%
torch.Size([64, 1, 28, 28])
Train loss: 18.37994 | Test loss: 2.28923, Test acc: 10.01%
torch.Size([64, 1, 28, 28])
Train loss: 20.67020 | Test loss: 2.28705, Test acc: 10.02%
torch.Size([64, 1, 28, 28])
Train loss: 22.96082 | Test loss: 2.28491, Test acc: 10.04%
torch.Size([64, 1, 28, 28])
Train loss: 25.25004 | Test loss: 2.28214, Test acc: 

In [14]:
all_X, all_y = [], []
for X, y in test_dataloader:
    X, y = X.to(device), y.to(device)
    all_X.append(X)  # Append each batch
    all_y.append(y)

X_test = torch.cat(all_X, dim=0)
y_test = torch.cat(all_y, dim=0)

final_preds = model(X_test)

print(classification_report(y_test.cpu().numpy(), final_preds.argmax(dim = 1).cpu().numpy()))

              precision    recall  f1-score   support

           0       0.78      0.85      0.81      1000
           1       0.97      0.97      0.97      1000
           2       0.66      0.88      0.75      1000
           3       0.90      0.84      0.87      1000
           4       0.78      0.73      0.75      1000
           5       0.98      0.93      0.96      1000
           6       0.72      0.49      0.58      1000
           7       0.91      0.96      0.93      1000
           8       0.95      0.96      0.96      1000
           9       0.95      0.95      0.95      1000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.85     10000
weighted avg       0.86      0.86      0.85     10000

